In [1]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import sys
sys.settrace
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from pylab import *
import struct
import array
import os
import glob
import h5py
from scipy.interpolate import griddata
import scipy.integrate as integrate


#import matplotlib as mpl
#mpl.use("pgf")
#pgf_with_custom_preamble = {
#    "font.family": "serif", # use serif/main font for text elements
#    "text.usetex": True,    # use inline math for ticks
#    "pgf.rcfonts": False,   # don't setup fonts from rc parameters
#    "pgf.preamble": [
#         r"\usepackage{units}",         # load additional packages
#         r"\usepackage{metalogo}",
#         r"\usepackage{unicode-math}",  # unicode math setup
#         r"\setmathfont{xits-math.otf}",
#         r"\setmainfont{DejaVu Serif}", # serif font via preamble
#         r"\usepackage{sfmath} \boldmath",
#         ]
#}

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
#plt.rcParams.update({
#    "text.usetex": True,
#    "font.family": "serif",
#    "font.serif": ["Palatino"],
#})

plt.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']
plt.rcParams['font.size'] = '20'

from importlib import reload

/var/folders/2n/ftypqy6s65s2j7krblnn49cw0000gq/T/ipykernel_64331/3945053999.py:46: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  plt.rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']


In [2]:
import athena_read

In [3]:
#This is the integral of blackbody from 0 to nu_t (h\nu/kT)
def FitBlackBody(nu_t):
    integral = 0.0;
    nu_2 = nu_t * nu_t;
    nu_3 = nu_t * nu_2;
    nu_7 = nu_2 * nu_2 * nu_3
    if nu_t < 1.8:
        integral = 0.051329911273422 * nu_3 -0.019248716727533 * nu_t * nu_3 + 0.002566495563671 * nu_2 * nu_3
        -3.055351861513195*1.e-5*nu_7
    elif nu_t < 18.6:
        exp_nu = exp(-nu_t);
        integral = -0.156915538762850 * exp_nu * (nu_3 + 2.69 * nu_2 + 6.714 * nu_t) + 1.000009331428801*(1- exp_nu);
    else:
        integral = 1.0 - 192.1*exp(-0.9014*nu_t)

    return integral;
#This is the blackbody spectrum for each frequency nu_t (h\nu/kT)
def BlackBody(nu, tr):
    nu_t=nu/tr
    exp_1=exp(nu_t)-1
    if nu_t < 1.e-5:
      exp_1=tr**3.0*nu_t**2.0
    else:
      exp_1=nu**3.0/exp_1

    spec=(15/np.pi**4.0)*exp_1

    return spec;

def BlackBodyNnu2(nu, tr):
    nu_t=nu/tr
    exp_1=exp(nu_t)-1
    if nu_t < 1.e-5:
        exp_1=tr**2.0*(nu_t-0.5*nu_t**2.0)
    else:
        exp_1=1/exp_1

    spec=exp_1

    return spec;
def ConvertJNNu2(bbj, nu_f):
    jnu4=bbj/nu_f**4
    log_jnu4=log10(jnu4)
    log_nnu2=1
    if log_jnu4 < -3.99793:
        log_nnu2 = 0.9971*log_jnu4+0.7464
    elif log_jnu4 >= -3.99793 and log_jnu4 < 2.73043:
        jsq = log_jnu4*log_jnu4
        log_nnu2 = 0.0003484*jsq*jsq+0.002327*jsq*log_jnu4-0.02137*jsq+0.8037*log_jnu4+0.3249
    else:
        log_nnu2 = 0.7506*log_jnu4+0.3773
    nnu2 = 10.0**log_nnu2*nu_f*nu_f*nu_f
    return nnu2
def ConvertJNNu2Wein(bbj, nu_f, tgas):
    nu_t=nu_f/tgas
    n_coef = 2.0+2.0*nu_t+nu_t*nu_t
    j_coef = 6.0+6.0*nu_t+3.0*nu_t*nu_t+nu_t*nu_t*nu_t
    nnu2 = bbj*np.pi**4.0*n_coef/(15.0*tgas*j_coef)
    return nnu2

In [4]:
def PlotProfile(datax, datay, xmin, xmax, ymin, ymax,  ylabel, label1, filename, xlabel='$r/r_g$', logscale=0, 
                xlogscale=0, datax1_2=None, datay1_2=None, datay1_3=None, datax2=None, datay2=None, datax2_2=None, 
                datay2_2=None, datay2_3=None, datax3=None, datay3=None, datax3_2=None, datay3_2=None, datay3_3=None, 
                datax4=None, datay4=None, datax4_2=None, datay4_2=None, datax5=None, datay5=None, label2='', 
                label2_2='',label3='', label4='', label5='',title=None,leg_loc=None):
    plots, axes = plt.subplots(figsize=(9,9),dpi=300)
    plt.xlabel(xlabel, size = 30)
    plt.ylabel(ylabel, size = 30)
    plt.subplots_adjust(left=0.15,right=0.88,top=0.9,bottom=0.1)
    plt.ylim([ymin,ymax])
    plt.xlim([xmin,xmax])
    if logscale > 0:
      axes.set_yscale('log')
    if xlogscale > 0:
      axes.set_xscale('log')
    if title is not None:
      plt.title(title,size=20)

    plt.plot(datax,datay,color='black',label=label1,linewidth=3.0)
    if datay1_2 is not None:
        if datax1_2 is not None:
            plt.scatter(datax1_2,datay1_2,s=80,facecolors='none', edgecolors='black')
        else:
            plt.scatter(datax1,datay1_2,s=80,facecolors='none', edgecolors='black')    
    if datay1_3 is not None:
        plt.plot(datax,datay1_3,color='black',linestyle='dashed',linewidth=4.0)
    if datay2 is not None:
        plt.plot(datax2,datay2,color='red',label=label2,linewidth=3.0)
    if datay2_2 is not None:
        if datax2_2 is not None:
            plt.plot(datax2,datay2,color='blue',label=label2_2,linewidth=3.0,linestyle='dashed')
        else:
            plt.plot(datax2,datay2,color='red',label=label2,linewidth=3.0)      
    if datay2_3 is not None:
        plt.plot(datax2,datay2_3,color='red',linestyle='dashed',linewidth=4.0)
    if datay3 is not None:
        plt.plot(datax3,datay3,color='green',label=label3,linewidth=3.0)
    if datay3_2 is not None:
        if datax3_2 is not None:
            plt.scatter(datax3_2,datay3_2,s=80,facecolors='none', edgecolors='green')
        else:
            plt.scatter(datax3_2,datay3_2,s=80,facecolors='none', edgecolors='green')          
    if datay3_3 is not None:
        plt.plot(datax3,datay3_3,color='green',linestyle='dashed',linewidth=4.0)
    if datay4 is not None:
        plt.plot(datax4,datay4,color='blue',label=label4,linewidth=3.0)
    if datay4_2 is not None:
        if datax4_2 is not None:
            plt.scatter(datax4_2,datay4_2,s=80,facecolors='none', edgecolors='blue')
        else:
            plt.scatter(datax4,datay4_2,s=80,facecolors='none', edgecolors='blue')
    if datay5 is not None:
      plt.plot(datax5,datay5,color=tableau20[0],label=label5,linewidth=2.0) 
    if leg_loc is not None:
      plt.legend(loc="best",bbox_to_anchor=leg_loc,frameon=False)
    axes.set_aspect('auto')
#    axes.yaxis.set_tick_params(labelsize=25)
#    axes.xaxis.set_tick_params(labelsize=25)
    plt.savefig(filename)
    plt.close(plots)


In [5]:
files_10=sorted(glob.glob('Tau_10/Data/thermal*athdf'))
files_30=sorted(glob.glob('Tau_30/Data/thermal*athdf'))
files_100=sorted(glob.glob('Tau_100/Data/thermal*athdf'))
files_300=sorted(glob.glob('Tau_300/Data/thermal*athdf'))

In [6]:
#for filename in files:
ang_file = open('Tau_10/Data/Rad_angles.txt', 'r')
num_file=size(files_10)
Lines = ang_file.readlines()
angle_line = [ line for line in Lines if  "tau_scheme" in line]
angle_line=angle_line[0]

prat_line = [ line for line in Lines if  "Prat" in line]
prat_line=prat_line[0]
prat_split=prat_line.split(" ")
Prat=float(prat_split[size(prat_split)-2])


crat_line = [ line for line in Lines if  "Crat" in line]
crat_line=crat_line[0]
Crat_split=crat_line.split(" ")
Crat=float(Crat_split[size(Crat_split)-2])


angle_index=Lines.index(angle_line)+1
location=Lines.index('fre   spec\n')
tot_line=size(Lines)
nfreq=tot_line-location-1
nu_grid=np.zeros(nfreq)
histories=np.zeros((num_file,52))
for i in range(nfreq):
    line=Lines[i+location+1].split(' ')[0]
    nu_grid[i]=float(line)

nu_center=np.zeros(nfreq-1)
nu_center[:]=(nu_grid[:-1]+nu_grid[1:])/2

nu_plot=np.zeros(nfreq)
nu_plot[:-1]=nu_center
nu_plot[nfreq-1]=nu_grid[nfreq-1]

#nu_center=np.append(nu_center,nu_grid[nfreq-1])
nang=location-angle_index
mu_x=np.zeros(nang)
mu_y=np.zeros(nang)
mu_z=np.zeros(nang)
weight=np.zeros(nang)
for n in range(nang):
    line=Lines[n+angle_index].split('   ')
    mu_x[n]=float(line[1])
    mu_y[n]=float(line[2])
    mu_z[n]=float(line[3])
    weight[n]=float(line[4].rstrip())

In [7]:
#get grid size from file[0]
filename=files_10[0]
with h5py.File(filename, 'r') as f:
    attributes = f.attrs.items()
    attrs = dict(attributes)
    level = f.attrs['MaxLevel']
    time = f.attrs['Time']
    subsample = False
data = athena_read.athdf(filename, level=level, subsample=subsample)

x1v=data['x1v']
meshsize=data['RootGridSize']
nx=meshsize[0]
dx=x1v[1]-x1v[0]
    
tgas=100
rho=np.zeros(nx)
sigma_s=np.zeros(nx)

er_spec_10=np.zeros((nfreq,nx))
er_profile_10=np.zeros(nx)
specific_spec_10=np.zeros((nfreq-1,nx))

er_spec_30=np.zeros((nfreq,nx))
er_profile_30=np.zeros(nx)
specific_spec_30=np.zeros((nfreq-1,nx))

er_spec_100=np.zeros((nfreq,nx))
er_profile_100=np.zeros(nx)
specific_spec_100=np.zeros((nfreq-1,nx))


er_spec_300=np.zeros((nfreq,nx))
er_profile_300=np.zeros(nx)
specific_spec_300=np.zeros((nfreq-1,nx))

In [8]:
#check the profile of the last file
num_file=size(files_10)
filename=files_10[num_file-1]
with h5py.File(filename, 'r') as f:
    attributes = f.attrs.items()
    attrs = dict(attributes)
    level = f.attrs['MaxLevel']
    time = f.attrs['Time']
    subsample = False
data = athena_read.athdf(filename, level=level, subsample=subsample)
rho=data['rho'][0,0,:]

for j in range(nfreq):
    varname='Er_'+str(j)
    er_spec_10[j,:]=data[varname][0,0,:]
for j in range(nfreq-1):
    specific_spec_10[j,:]= er_spec_10[j,:]/(nu_grid[j+1]-nu_grid[j])
    
for i in range(nx):    
    er_profile_10[i]=np.sum(er_spec_10[:,i])

varname='Sigma_s_0'
sigma_s[:]=data[varname][0,0,:]

##-------------------------------------

num_file=size(files_30)
filename=files_30[num_file-1]
with h5py.File(filename, 'r') as f:
    attributes = f.attrs.items()
    attrs = dict(attributes)
    level = f.attrs['MaxLevel']
    time = f.attrs['Time']
    subsample = False
data = athena_read.athdf(filename, level=level, subsample=subsample)

for j in range(nfreq):
    varname='Er_'+str(j)
    er_spec_30[j,:]=data[varname][0,0,:]
for j in range(nfreq-1):
    specific_spec_30[j,:]= er_spec_30[j,:]/(nu_grid[j+1]-nu_grid[j])
    
for i in range(nx):    
    er_profile_30[i]=np.sum(er_spec_30[:,i])

##-------------------------------------

num_file=size(files_100)
filename=files_100[num_file-1]
with h5py.File(filename, 'r') as f:
    attributes = f.attrs.items()
    attrs = dict(attributes)
    level = f.attrs['MaxLevel']
    time = f.attrs['Time']
    subsample = False
data = athena_read.athdf(filename, level=level, subsample=subsample)

for j in range(nfreq):
    varname='Er_'+str(j)
    er_spec_100[j,:]=data[varname][0,0,:]
for j in range(nfreq-1):
    specific_spec_100[j,:]= er_spec_100[j,:]/(nu_grid[j+1]-nu_grid[j])
    
for i in range(nx):    
    er_profile_100[i]=np.sum(er_spec_100[:,i])
    
##-------------------------------------

num_file=size(files_300)
filename=files_300[num_file-1]
with h5py.File(filename, 'r') as f:
    attributes = f.attrs.items()
    attrs = dict(attributes)
    level = f.attrs['MaxLevel']
    time = f.attrs['Time']
    subsample = False
data = athena_read.athdf(filename, level=level, subsample=subsample)

for j in range(nfreq):
    varname='Er_'+str(j)
    er_spec_300[j,:]=data[varname][0,0,:]
for j in range(nfreq-1):
    specific_spec_300[j,:]= er_spec_300[j,:]/(nu_grid[j+1]-nu_grid[j])
    
for i in range(nx):    
    er_profile_300[i]=np.sum(er_spec_300[:,i])
 

In [9]:
print(er_profile_10)

[0.9274009  0.91466078 0.8895267  0.8688469  0.84646342 0.82469571
 0.80273015 0.78090348 0.75913073 0.73748018 0.71596395 0.69461371
 0.67345318 0.65250796 0.63180178 0.61135778 0.5911979  0.57134301
 0.55181275 0.53262552 0.51379839 0.4953471  0.47728597 0.45962795
 0.44238453 0.42556579 0.40918037 0.39323555 0.37773718 0.3626898
 0.34809666 0.33395983 0.32028022 0.3070577  0.2942912  0.28197868
 0.27011717 0.25870282 0.24773079 0.23719538 0.22709002 0.21740736
 0.20813937 0.1992774  0.19081233 0.18273463 0.17503445 0.1677017
 0.16072608 0.15409715 0.14780437 0.14183711 0.13618469 0.1308364
 0.12578153 0.12100937 0.11650923 0.11227049 0.10828257 0.10453499
 0.1010174  0.09771954 0.09463132 0.09174283 0.08904433 0.08652631
 0.08417945 0.0819947  0.07996326 0.07807659 0.07632642 0.07470478
 0.07320399 0.07181665 0.07053569 0.06935431 0.06826604 0.0672647
 0.06634439 0.06549954 0.06472484 0.06401529 0.06336615 0.06277294
 0.06223146 0.06173777 0.06128815 0.06087914 0.06050748 0.06017014

In [12]:
plt.plot(nu_center,specific_spec[:,nx-1])
plt.plot(nu_center,spe_fr_spec[:,nx-1])
plt.xscale('log')
plt.yscale('log')
plt.ylim(1.e-7,1)

NameError: name 'specific_spec' is not defined

In [10]:
ylabel='$E_r(\\tilde{\\nu})$'
filename='compton_disk_tau.pdf'
xlabel="$\\tilde{\\nu}$"
label2='$\\tau=100$'
label3='$\\tau=30$'
label4='$\\tau=10$'
#print(histories[:,0])
PlotProfile(nu_center,specific_spec_300[:,nx-1],3.e-2, 550, 1.e-10, 5.e-2, ylabel, '$\\tau=300$',
            filename, xlabel,logscale=1,xlogscale=1,
            datax2=nu_center,datay2=specific_spec_100[:,nx-1],label2=label2,
            datax3=nu_center,datay3=specific_spec_30[:,nx-1],label3=label3,
            datax4=nu_center,datay4=specific_spec_10[:,nx-1],label4=label4,leg_loc=(0.32,0.3))

In [31]:
print(er_profile_10[nx-1],er_profile_30[nx-1],er_profile_100[nx-1],er_profile_300[nx-1])
print(er_spec_10[nfreq-1,nx-1],er_spec_30[nfreq-1,nx-1],er_spec_100[nfreq-1,nx-1],er_spec_300[nfreq-1,nx-1])

0.053829888818609416 0.026091590939543607 0.13875776601869738 0.20862580334917438
5.659393328972794e-20 1.5075755754878628e-07 0.01782192289829254 0.05544385686516762


In [2]:
3/5.94e3

0.000505050505050505